In [ ]:
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

In [39]:
from clients.gpt_database_logger import GPTDatabaseLogger

GPT_DB_LOGGER_PATH = 'sqlite:///prompt_versions.db'

db_logger = GPTDatabaseLogger(GPT_DB_LOGGER_PATH)

In [ ]:
import os
from email import message_from_file


def process_directory(directory_path):
    email_data = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.eml'):
            with open(os.path.join(directory_path, file_name), 'rb') as f:
                email_info = extract_email_data(f)
                email_data.append(email_info)
    
    # Sort the emails based on your criteria, e.g., timestamp
    sorted_emails = sorted(email_data, key=lambda x: x['timestamp'])

    return sorted_emails


sorted_emails = process_directory(path)

for email in sorted_emails:
    print(email)


In [22]:
from bs4 import BeautifulSoup
import copy
import re
import json
import hashlib
import email
from email.policy import default

In [11]:
from email import policy
from email.parser import BytesParser
from email.message import EmailMessage
import os


def extract_email_data(eml_file_path) -> dict:
    """
    Extracts data from an .eml file.

    Args:
    eml_file_path (str): Path to the .eml file.

    Returns:
    dict: Extracted email data including sender, recipient, subject, date, and body.
    """

    with open(eml_file_path, 'rb') as f:
        msg: EmailMessage = BytesParser(policy=policy.default).parse(f)

    # Extracting basic details
    email_data = {
        "from": msg["From"],
        "to": msg["To"],
        "subject": msg["Subject"],
        "date": msg["Date"]
    }

    # Extracting the body of the email
    if msg.is_multipart():
        parts = [part.get_payload(decode=True) for part in msg.walk() if part.get_content_type() == 'text/plain']
        body = ''.join(str(part, 'utf-8') for part in parts)
    else:
        body = str(msg.get_payload(decode=True), 'utf-8')

    email_data["body"] = body.strip()

    return email_data
    

In [12]:
email_data = extract_email_data('./deals_html/413460.html')

In [26]:
def extract_messages_from_raw_html(html_content: str):
    soup = BeautifulSoup(html_content, "html.parser")
    messages = []
    clone_body = copy.copy(soup)

    for nested_blockquote in clone_body.find_all("blockquote"):
        nested_blockquote.decompose()

    messages.append(clone_body.get_text(strip=True))
    blockquotes = soup.find_all("blockquote")

    for blockquote in blockquotes:
        clone = copy.copy(blockquote)

        for nested_blockquote in clone.find_all("blockquote"):
            nested_blockquote.decompose()

        messages.append(clone.get_text(strip=True))

    return messages


def extract_messages_from_raw_html(html_content: str):
    """
    Метод парсит сырой HTML переписки и возвращает сообщения по убыванию по дате
    (т.е с последнего сообщения к первому)
    """
    soup = BeautifulSoup(html_content, "html.parser")
    messages = []
    clone_body = copy.copy(soup)

    for nested_blockquote in clone_body.find_all("blockquote"):
        nested_blockquote.decompose()

    messages.append(clone_body.get_text(strip=True))
    blockquotes = soup.find_all("blockquote")

    for blockquote in blockquotes:
        clone = copy.copy(blockquote)

        for nested_blockquote in clone.find_all("blockquote"):
            nested_blockquote.decompose()

        messages.append(clone.get_text(strip=True))

    from_end_to_start_messages = messages
    return from_end_to_start_messages


In [17]:
with open('./deals_html/413460.html', 'rb') as f:
    html = f.read()

In [24]:
messages = extract_messages_from_raw_html(html)

In [30]:
print(messages[3])

<@font-face {font-family:Calibri; panose-1:2 15 5 2 2 2 4 3 2 4;}p.MsoNormal, li.MsoNormal, div.MsoNormal {margin:0cm; margin-bottom:.0001pt; font-size:11.0pt; font-family:"Calibri",sans-serif; mso-fareast-language:RU-US;}a:link, span.MsoHyperlink {mso-style-priority:99; color:blue; text-decoration:underline;}a:visited, span.MsoHyperlinkFollowed {mso-style-priority:99; color:purple; text-decoration:underline;}span.EmailStyle17 {mso-style-type:personal-compose; font-family:"Calibri",sans-serif; color:windowtext;}.MsoChpDefault {mso-style-type:export-only; font-family:"Calibri",sans-serif; mso-fareast-language:RU-US;}Mit
            freundlichen Grüßen / Best regards,Kristine GergaiaSales
          ManagerFAMAGA
          Group OHG23560, Lubeck, Hinter den
          Kirschkaten 83+49 (451) 70 36 49 97 (134)https://famaga.de/-------- Forwarded Message --------Subject:10001-03Date:Mon, 12 Jun 2023 12:25:59 +0000From:eutradesales<eutradesales@proton.me>To:Kristine Gergaia<kg1@famaga.de>I
  

In [31]:
import re

# Function to remove HTML tags from a string
def strip_html(html_string):
    clean_text = re.sub(r'<[^>]+>', '', html_string)
    return clean_text

In [32]:
strip_html(messages[3])

'To:Kristine GergaiaI\r\r\n            have a new request for you. give me a quote please.And also please send the companies with which you work.\r\r\n            It is quite possible that we could work with you in a larger\r\r\n            volume.Since\r\r\n              I am looking for a lot of products on the Internet, and\r\r\n              very often your site opens up to me.Also,\r\r\n                if you want, you can send me your WhatsApp number, for\r\r\n                faster communication.END\r\r\n                ARMATUREN SK711067/PA (1 pc)END ARMATUREN ZA64-EE63 EE620632 (1 pc)END ARMATUREN ZA310066-EE620852/AX\xa0 (1 pc)'

In [ ]:
offer_message = "Please feel free to contact us if you need any further information"
msgs_without_offers = [msg for msg in messages if offer_message not in msg]

for msg in msgs_without_offers:
    print(msg)
    print('\n\n')



In [44]:
msgs_without_offers[-1].strip()

'-------- Forwarded\r\r\n                                                Message --------Subject:Fwd:\r\r\n                                                        9089,7951-7954,\r\r\n                                                        6553,6554Date:Tue, 30 May\r\r\n                                                        2023 17:01:54\r\r\n                                                        +0300From:info@famaga.de<info@famaga.de>To:sales@famaga.de<sales@famaga.de>--------\r\r\n                                                  Weitergeleitete\r\r\n                                                  Nachricht --------Betreff:9089,7951-7954,\r\r\n                                                          6553,6554Datum:Tue, 30 May\r\r\n                                                          2023 13:53:48\r\r\n                                                          +0000Von:eutradesales<eutradesales@proton.me>An:info@famaga.de<info@famaga.de>Dear\r\r\n                            

In [45]:
response = db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Try to extract from text brand name, amount, detail name, part number from the text. Also recognize country by text.
<<<>>>
{msgs_without_offers[-1].strip()}
<<<>>>

If you cannot recognize specified parameters please put `null` value.

---

Your response should be a list of comma separated values, eg: `foo, bar, baz`

The output should be a markdown code snippet formatted in the following adr, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{{
    "parts": [
        {{
           "amount": int // This is the amount of details
           "brand_name": string // This is thr brand  name of detail
           "part_number": string // This is the part number of detail
        }}
    ],
    "client": {{
        "country": string // This is the country of detail,
        "domain": string // customer company domain
        "email": string // customer email,
        "office_country": string // country of customer office
    }}
}}
```
    """}                             
    ])

<>:33: SyntaxWarning: invalid escape sequence '\`'
<>:33: SyntaxWarning: invalid escape sequence '\`'
/var/folders/x2/0f74_mmn0ts5qbnsn_wp241w0000gn/T/ipykernel_78512/3047214954.py:33: SyntaxWarning: invalid escape sequence '\`'
  """}


```json
{
    "parts": [
        {
           "amount": 1,
           "brand_name": "K-Tron",
           "part_number": "9473-41073"
        },
        {
           "amount": 2,
           "brand_name": "K-Tron",
           "part_number": "0000004180"
        },
        {
           "amount": 2,
           "brand_name": "K-Tron",
           "part_number": "0000025153"
        },
        {
           "amount": 2,
           "brand_name": "K-Tron",
           "part_number": "0000005108"
        },
        {
           "amount": 2,
           "brand_name": "K-Tron",
           "part_number": "0000038727"
        },
        {
           "amount": 2,
           "brand_name": "K-TRON",
           "part_number": "MTR,SA STEPPER 1500RPM 170V 2.7A, 7518"
        },
        {
           "amount": 1,
           "brand_name": "K-TRON",
           "part_number": "BRUSH,SEAL (OEM HU), 527"
        }
    ],
    "client": {
        "country": "Latvia",
        "domain": "proton.me",
        "email": "

Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [22]:
from bs4 import BeautifulSoup, Tag

def build_html_structure(html):
    soup = BeautifulSoup(html, 'html.parser')
    max_depth = 20  # Estimate the maximum depth of your HTML
    # Use 2 spaces for indentation, adjust number as needed
    indentations = ['  ' * i for i in range(max_depth + 1)]  # Precompute indentations using 2 spaces

    # Recursive function to generate HTML-like string without text nodes
    def generate_html_structure(element, depth=0):
        if isinstance(element, Tag):
            indent = indentations[depth]
            class_attr = f'class="{element["class"]}"' if 'class' in element.attrs else ''
            tag_open = f"{indent}<{element.name} {class_attr.strip()}>"

            # Gather all child tags for processing
            child_tags = [child for child in element.children if isinstance(child, Tag)]
            
            if len(child_tags) == 1:
                # Single child: Use single-line format
                child_html = generate_html_structure(child_tags[0], depth + 1).strip()
                return f"{tag_open}{child_html}</{element.name}>"
            elif child_tags:
                # Multiple children: Use multiline format with children on new lines
                children_html = ''.join(generate_html_structure(child, depth + 1) for child in child_tags)
                tag_close = f"{indent}</{element.name}>"
                return f"{tag_open}\n{children_html}\n{tag_close}"
            else:
                # No children: Close tag on the same line
                return f"{tag_open}</{element.name}>"

        return ''  # Ignore non-tag elements and text

    # Start from the root of the document and skip the [document] tag
    return ''.join(generate_html_structure(child) for child in soup.children)

In [192]:
exmpl_file_1 = './deals_html/discount/380760.html'
exmpl_file_2 = './deals_html/discount/376107.html'
exmpl_file_3 = './deals_html/discount/382491.html'
exmpl_file_4 = './deals_html/discount/397719.html'
exmpl_file_5 = './deals_html/391303.html'


exmpl_file = exmpl_file_4

with open(exmpl_file, 'rb') as f:
    html = f.read()

In [ ]:

structured_html = build_html_structure(html)
print(structured_html)

In [188]:
soup = BeautifulSoup(html, "html.parser")
body = soup.find('body')

In [ ]:
soup

In [193]:
from typing import List

specific_style = "border:none;border-top:solid"


def get_messages_from_html_file(file_path: str):
    with open(exmpl_file, 'rb') as f:
        html = f.read()
        return split_email_html_on_messages(html)
        
def split_email_html_on_messages(email_html) -> List[str]:
    soup = BeautifulSoup(html, "html.parser")
    body = soup.find('body')
    return get_element_message(body)

keys_to_check = ['From:', 'Sent:', 'To:', 'Subject:']

# Function to check presence of keys
def check_presence_of_keys(element, keys):
    text = element.get_text()  # Extract all text from the HTML
    missing_keys = [key for key in keys if key not in text]
    return missing_keys


def get_element_messages(element) -> List[str]:
    print(element.name)
    child_elements = [desc for desc in element.children if desc.name in ['p', 'div', 'table', 'blockquote']]
    messages = []

    message = ''

    def _append_message(msg: str):
        if msg.strip():
            messages.append(msg)
        
    for child in child_elements:
        if child.name == 'div':
            blockquote = child.find('blockquote')
            if blockquote:
                _append_message(message)
                message = ''
                messages += get_element_messages(child)
            elif hasattr(child, 'style') and child.get('style') and specific_style in child.get('style'):
                _append_message(message)
                message = ''
            else:
                message += child.get_text(strip=True)
        elif child.name == 'p':
            if hasattr(child, 'class') and child.get('class') and child.get('class')[0] == 'moz-forward-container':
                _append_message(message)
                message = ''
            elif not check_presence_of_keys(child, keys_to_check):
                _append_message(message)
                message = ''
                message += child.get_text(strip=True) 
            else:
                message += child.get_text(strip=True)
        elif child.name == 'blockquote':
            _append_message(message)
            message = ''
            messages += get_element_messages(child)
        elif child.name == 'table':
            blockquote = child.find('blockquote')
            if blockquote:
                print('Table blockquote')
                _append_message(message)
                message = ''
                messages += get_element_messages(blockquote)
            else:
                message += child.get_text(strip=True)
        else:
            message += child.get_text(strip=True)

    _append_message(message)

    return messages

soup = BeautifulSoup(html, "html.parser")
root_element = soup.find('body') if soup.find('body') else soup
messages = get_element_messages(root_element)

[document]
div
div
blockquote
blockquote
blockquote
div
div
div
div
div
div
div
div
div
blockquote


In [195]:
for idx, msg in enumerate(messages):
    print(f'Message {len(messages) - idx}:\n```\n{msg}\n```\n\n')

Message 8:
```
Hello Ali,Hope this email finds you well.Please be informed that we had submitted our offer to the end user and got feedback from them that our prices were on the higher side by 45% from the prices other vendors quoted to them.Hence we request you to please recheck the prices and help us so we will be able to win this bid.
```


Message 7:
```
On Thu, 13 Apr 2023 at 21:06, <maa@famaga.de> wrote:
```


Message 6:
```
Dear Sir/ Madam,I would like to follow up on quotation, which was sent in response to your inquiry.Our company would be glad to provide any additional information and make every effort to fulfill your requirements and receive an order!Please take into consideration, that our portfolio not limited to this brand only.One of our distinctive features is close cooperation with various manufacturers from Europe, United States, Japan and other countries around the world.Famaga Group OHG works directly with them, avoiding a long chain of agents, which allows us to pr

In [64]:
signature_div = soup.find('div', class_='moz-signature')


In [68]:
signature_div

<div class="moz-signature">
<style><!--@font-face {font-family:"Cambria Math"; panose-1:2 4 5 3 5 4 6 3 2 4;}@font-face {font-family:Calibri; panose-1:2 15 5 2 2 2 4 3 2 4;}/* Style Definitions */p.MsoNormal, li.MsoNormal, div.MsoNormal {margin:0cm; margin-bottom:.0001pt; font-size:11.0pt; font-family:"Calibri",sans-serif; mso-fareast-language:RU-US;}a:link, span.MsoHyperlink {mso-style-priority:99; color:blue; text-decoration:underline;}a:visited, span.MsoHyperlinkFollowed {mso-style-priority:99; color:purple; text-decoration:underline;}span.EmailStyle17 {mso-style-type:personal-compose; font-family:"Calibri",sans-serif; color:windowtext;}.MsoChpDefault {mso-style-type:export-only; font-family:"Calibri",sans-serif; mso-fareast-language:RU-US;}--></style><!--[if gte mso 9]><xml><o:shapedefaults v:ext="edit" spidmax="1027" /></xml><![RUdif]--><!--[if gte mso 9]><xml><o:shapelayout v:ext="edit"><o:idmap v:ext="edit" data="1" /></o:shapelayout></xml><![RUdif]--><strong><span style="color:

In [67]:
signature_div.get_text()

'\nMit\r\r\n          freundlichen Grüßen / Best regards, \r\r\n          Vlad Pugach\nAftersales\nFAMAGA\r\r\n        Group OHG\n\n23560, Lubeck, Hinter den Kirschkaten 83\n  +\n  https://famaga.de/\r\r\n          '